In [3]:
from math import sqrt
from joblib import Parallel, delayed
import numpy as np
from tqdm import tqdm
from glob import glob
from os.path import join
from skimage import io

class Eval(object):
    """Eval traffic sign dataset """

    def __init__(self):
        pass

    def read_dataset(self, path):
        """ Read dataset images filename
        :param path: Path were the dataset is stored
        :returns: A dictionary with the dataset filenames
        """
        # train
        train_l = glob(join(path,'train','*.jpg'))
        dataset = [{'train':t, 'gt':t.replace('train','train_masks')} for t in train_l]
        
        return dataset
    @staticmethod 
    def segment_image(im, color_space):
       
        """Segments an image using a given color space
        :param im: Input image (x,x,3)
        :param color_space: Color space required for segmentation
        :returns: uint8 mask. 255 true, 0 False
        """

        
        ## Add Segmentation code HERE
        if color_space == 'RGB':
            #Receive two parameters, in case there is no color we look for in the image will return a black color, otherwise calculate the average
            def means(arr, ch):
                if arr.size == 0:
                    return 0, 0, 0, 0, 0, 0
                else:
                    x1 = int(np.mean(arr[:, 0]))
                    x2 = int(np.mean(arr[:, 1]))
                    x3 = int(np.mean(arr[:, 2]))
                # Paint the true ones and not paint the false ones while fulfilling the ranks of the average
                if ch == 'r':
                    return x1 - 50, x1 + 50, x2 - 30, x2 + 30, x3 - 30, x3 + 30
                if ch == 'b':
                    return x1 - 40, x1 + 40, x2 - 40, x2 + 40, x3 - 60, x3 + 60
                if ch == 'y':
                    return x1 - 30, x1 + 30, x2 - 30, x2 + 30, x3 - 30, x3 + 30
            #Scroll through the image looking for the positions where the conditional is fulfilled, for the color red, blue and yellow
            red = ((im[:, :, 0].astype(np.int16) - im[:, :, 1].astype(np.int16)) > 70) & \
                   ((im[:, :, 0].astype(np.int16) - im[:, :, 2].astype(np.int16)) > 70)
            #arr_r. keeps in a list the values ​​that comply with the previous condition
            arr_r = im[red]
            blue = ((im[:, :, 2].astype(np.int16) - im[:, :, 0].astype(np.int16)) > 60) & \
                   ((im[:, :, 2].astype(np.int16) - im[:, :, 1].astype(np.int16)) > 60)
            arr_a = im[blue]
            yellow = (((im[:, :, 0].astype(np.int16) - im[:, :, 1].astype(np.int16)) < 70) &\
                     ((im[:, :, 1].astype(np.int16) - im[:, :, 0].astype(np.int16)) <np.absolute(35))) & \
                     (((im[:, :, 0].astype(np.int16) - im[:, :, 2].astype(np.int16)) > 70) &
                     ((im[:, :, 1].astype(np.int16) - im[:, :, 2].astype(np.int16)) > 70))
            arr_y = im[yellow]
            #we get the calculated values ​​of the function means
            rmin, rmax, gmin, gmax, bmin, bmax = means(arr_r, 'r')
            rmin1, rmax1, gmin1, gmax1, bmin1, bmax1 = means(arr_a, 'b')
            rmin2, rmax2, gmin2, gmax2, bmin2, bmax2 = means(arr_y, 'y')
            #Segment red, blue, yellow and white. Based on the values ​​obtained from the function
            mask = (im[:, :, 0] > rmin) & (im[:, :, 0] < rmax) & \
                   (im[:, :, 1] >= gmin) & (im[:, :, 1] < gmax) & \
                   (im[:, :, 2] >= bmin) & (im[:, :, 2] < bmax) | \
                   (im[:, :, 0] >= 0) & (im[:, :, 0] < rmax1) & \
                   (im[:, :, 1] >= 0) & (im[:, :, 1] < gmax1) & \
                   (im[:, :, 2] > bmin1) & (im[:, :, 2] < bmax1) | \
                   (im[:, :, 0] > 210) & (im[:, :, 0] < rmax2) & \
                   (im[:, :, 1] > 210) & (im[:, :, 1] < gmax2) & \
                   (im[:, :, 2] > bmin2) & (im[:, :, 2] < bmax2) | ( \
                   (im[:, :, 0] >= 210) & (im[:, :, 0] <= 255) & (im[:, :, 1] >= 195) & \
                   (im[:, :, 1] <= 255) & (im[:, :, 2] >= 190) & (im[:, :, 2] <= 255))
            #converts the boolean mask to uint8
            mask = np.uint8(mask)
            #Assign the false to 0 and the true to 255 to see the segmentation
            """cero = (mask[:] == 0)
            mask[cero] = [0]
            blanco = (mask[:] == 1)
            mask[blanco] = [255]"""

        return mask
        ## Code ends
    @staticmethod
    def eval_mask(gt, mask):
        
        """ Eval mask
        :param gt: Groundtruth mask (binary values)
        :param mask: Obtained mask (binary values)
        :returns: tp, tn, fp, fn in pixels
        """
        tp_m = (gt==1) & (mask==1)
        tn_m = (gt==0) & (mask==0)
        fp_m = (gt==0) & (mask==1)
        fn_m = (gt==1) & (mask==0)
        return (tp_m.sum(), tn_m.sum(), fp_m.sum(), fn_m.sum())

    @staticmethod
    def process_image(ip):
        im=io.imread(ip['train'])
        gt=io.imread(ip['gt'])
        mask = Eval.segment_image(im, color_space)
        (tp, tn, fp, fn) = Eval.eval_mask((gt==255), mask)
        cal_metric=Eval.calc_metrics(tp,tn,fp,fn)
        return cal_metric
    @staticmethod
    def calc_metrics(tp,tn,fp,fn):
        TP += tp
        TN += tn
        FP += fp
        FN += fn
        precision = TP/(TP+FP)
        accuracy = (TP + TN)/ (TP + TN + FP +FN)
        recall = TP/(TP+FN)
        print('Precision = ', precision)
        print('Accuracy = ',accuracy)
        print('Recall = ', recall)
        print('F-measure = ', 2 * (precision * recall)/(precision+recall))
    @staticmethod   
    def process_dataset(dataset, color_space):
        
        """ Process the full dataset
        :param dataset: Dicctionary that contains all dataset filenames
        :param color_space: Color space required for segmentation
        :returns: None
        """
        TP = 0
        TN = 0
        FP = 0
        FN = 0
        Parallel(n_jobs=2)(delayed(Eval.process_image)(ip) for ip in dataset)
        """ for ip in tqdm(dataset):
            im = io.imread(ip['train'])
            gt = io.imread(ip['gt'])
            mask = self.segment_image(im, color_space)
            (tp, tn, fp, fn) = self.eval_mask((gt==255), mask)
           
        TP += tp
        TN += tn
        FP += fp
        FN += fn
        precision = TP/(TP+FP)
        accuracy = (TP + TN)/ (TP + TN + FP +FN)
        recall = TP/(TP+FN)
        print('Precision = ', precision)
        print('Accuracy = ',accuracy)
        print('Recall = ', recall)
        print('F-measure = ', 2 * (precision * recall)/(precision+recall))
        """ 

    def eval(self, path, color_space='RGB'):
        """ Segmentation evaluation on training dataset
        :param path: Dataset path
        :param color_space: Select segmentation color space
        :returns: None
        """
        dataset = self.read_dataset(path)
        self.process_dataset(dataset, color_space)

path="/home/cesar/Desktop/dataset/"
cvhelper = Eval()
image_list = cvhelper.eval(path)

JoblibNameError: JoblibNameError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/usr/lib/python3.5/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    179         sys.exit(msg)
    180     main_globals = sys.modules["__main__"].__dict__
    181     if alter_argv:
    182         sys.argv[0] = mod_spec.origin
    183     return _run_code(code, main_globals, None,
--> 184                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.5/dist-packages/ipykernel_launcher.py')
    185 
    186 def run_module(mod_name, init_globals=None,
    187                run_name=None, alter_sys=False):
    188     """Execute a module's code without importing it

...........................................................................
/usr/lib/python3.5/runpy.py in _run_code(code=<co